In [1]:
import numpy as np
import pandas as pd
import xarray as xr
import glob
import matplotlib
import matplotlib.pyplot as plt

In [2]:
### import some analysis functions we wrote for this project
import sys ; sys.path.append("..")
from ppe_tools.analysis import top_n

In [3]:
mainvars = ['FAREA_BURNED','FGEV','QOVER','QDRAI','SOILWATER_10CM','SNOWDP',
             'TSOI_10CM','TLAI','FSR','FSH','SCA_JJA','SCA_DJF','ALTMAX']
fluxvars = ['GPP','NPP','AR','HR','COL_FIRE_CLOSS','FCEV','FCTR']
cpools   = ['TOTVEGC','TOTSOMC_1m']



In [11]:
datavars = mainvars+[f+'_TLAI' for f in fluxvars]+[c+'_NPP' for c in cpools]
ens_name = 'CTL2010'
domain   = 'global'
nx       = 5
op       = 'mean'

In [51]:
def comp_lists(d1,d2,name1,name2):
    l1 = [x for x in d1]
    l2 = [x for x in d2]

    nc1 = [len(x) for x in l1]
    nc2 = [len(x) for x in l2]
    nc = max(max(nc1),max(nc2))


    s1 = set(l1)
    s2 = set(l2)

    both   = sorted(list(s1.intersection(s2)))
    uniqs1 = sorted(list(s1-s2))
    uniqs2 = sorted(list(s2-s1))

    if len(uniqs1)<len(uniqs2):
        uniqs1 = uniqs1+['']*(len(uniqs2)-len(uniqs1))
    else:
        uniqs2 = uniqs2+['']*(len(uniqs1)-len(uniqs2))


    print(3*' ',name1.ljust(nc+2),name2)
    print((2*nc+5)*'-') 
    i=0
    for p in both:
        i+=1
        print(str(i).ljust(3),p.ljust(nc+2),p)

    print((2*nc+5)*'-')

    for p1,p2 in zip(uniqs1,uniqs2):
        i+=1
        print(str(i).ljust(3),p1.ljust(nc+2),p2)


In [21]:
def make_list(datavars,ens_name,domain,nx,op):
    out = dict()
    for v in datavars:
        f  = './data/'+ens_name+'_'+v+'_'+domain+'.nc'
        ds = xr.open_dataset(f)
        dv = v+'_'+op
        if dv in ds.data_vars:
            da = ds[dv]
            p = ds.param
            m = ds.minmax
            xmins,xmaxs,pvals = top_n(da,nx,p,m)
            for p in pvals:
                if p not in out:
                    out[p] = [dv]
                else:
                    out[p] = [*out[p],dv]
    return out
    

In [56]:
cgm5 = make_list(datavars,'CTL2010','global',5,'mean')
cgi6 = make_list(datavars,'CTL2010','global',6,'iav')
comp_lists(cgm5,cgi6,'CTL-glob-mean','CTL-glob-iav')

    CTL-glob-mean                  CTL-glob-iav
-------------------------------------------------------------
1   FUN_fracfixers                 FUN_fracfixers
2   baseflow_scalar                baseflow_scalar
3   clay_pf                        clay_pf
4   cv                             cv
5   d_max                          d_max
6   decomp_depth_efolding          decomp_depth_efolding
7   dleaf                          dleaf
8   fff                            fff
9   frac_sat_soil_dsl_init         frac_sat_soil_dsl_init
10  fsr_pft                        fsr_pft
11  jmaxb0                         jmaxb0
12  jmaxb1                         jmaxb1
13  kmax                           kmax
14  leaf_long                      leaf_long
15  liq_canopy_storage_scalar      liq_canopy_storage_scalar
16  maximum_leaf_wetted_fraction   maximum_leaf_wetted_fraction
17  medlynintercept                medlynintercept
18  medlynslope                    medlynslope
19  om_frac_sf                     om

In [57]:
datavars = mainvars+cpools+fluxvars
cgm5nn = make_list(datavars,'CTL2010','global',5,'mean')
comp_lists(cgm5,cgm5nn,'CTL-glob-mean','CTL-glob-mean-not-normalized')

    CTL-glob-mean                  CTL-glob-mean-not-normalized
-------------------------------------------------------------
1   FUN_fracfixers                 FUN_fracfixers
2   accum_factor                   accum_factor
3   baseflow_scalar                baseflow_scalar
4   ceta                           ceta
5   clay_pf                        clay_pf
6   cv                             cv
7   d_max                          d_max
8   decomp_depth_efolding          decomp_depth_efolding
9   dleaf                          dleaf
10  fff                            fff
11  frac_sat_soil_dsl_init         frac_sat_soil_dsl_init
12  fsr_pft                        fsr_pft
13  jmaxb0                         jmaxb0
14  jmaxb1                         jmaxb1
15  kmax                           kmax
16  leaf_long                      leaf_long
17  liq_canopy_storage_scalar      liq_canopy_storage_scalar
18  lmr_intercept_atkin            lmr_intercept_atkin
19  maximum_leaf_wetted_fraction   maxim

In [135]:
datavars = mainvars+[f+'_TLAI' for f in fluxvars]+[c+'_NPP' for c in cpools]
datavars.remove('SCA_JJA')
datavars.remove('SCA_DJF')
datavars.remove('ALTMAX')
biomes = 1+np.arange(9)
nx = 3
cbm3 = make_list_biome(datavars,nx,biomes)

In [136]:
comp_lists(cgm5,cbm3,'CTL-glob-mean','CTL-biome-mean')

    CTL-glob-mean                  CTL-biome-mean
-------------------------------------------------------------
1   FUN_fracfixers                 FUN_fracfixers
2   accum_factor                   accum_factor
3   baseflow_scalar                baseflow_scalar
4   clay_pf                        clay_pf
5   cv                             cv
6   d_max                          d_max
7   decomp_depth_efolding          decomp_depth_efolding
8   dleaf                          dleaf
9   fff                            fff
10  frac_sat_soil_dsl_init         frac_sat_soil_dsl_init
11  froot_leaf                     froot_leaf
12  jmaxb0                         jmaxb0
13  jmaxb1                         jmaxb1
14  kmax                           kmax
15  leaf_long                      leaf_long
16  liq_canopy_storage_scalar      liq_canopy_storage_scalar
17  lmr_intercept_atkin            lmr_intercept_atkin
18  lmrha                          lmrha
19  maximum_leaf_wetted_fraction   maximum_leaf_we

In [144]:
for i,b in enumerate(ds.biome_name.values):
    print(str(i).ljust(3),b)

0   Ice sheet
1   Tropical rain forest
2   Tropical seasonal forest/savanna
3   Subtropical desert
4   Temperate rain forest
5   Temperate seasonal forest
6   Woodland/shrubland
7   Temperate grassland/desert
8   Boreal forest
9   Tundra


In [141]:
bps = set(cbm3)-set(cgm5)
for p in bps:
    print(p.ljust(20),cbm3[p])

q10_mr               {'AR_TLAI': [1, 2, 3], 'TOTVEGC_NPP': [3], 'TOTSOMC_1m_NPP': [1, 2, 9]}
n_baseflow           {'QOVER': [1]}
e_ice                {'SNOWDP': [5, 8, 9]}
wind_snowcompact_fact {'SNOWDP': [2]}
kn_nonmyc            {'GPP_TLAI': [9], 'AR_TLAI': [9]}
zlnd                 {'SNOWDP': [1], 'FSH': [3]}
krmax                {'QDRAI': [3, 7]}
fun_cn_flex_b        {'SNOWDP': [1]}
minpsi_hr            {'TOTSOMC_1m_NPP': [3, 7, 8]}
lmrhd                {'GPP_TLAI': [4, 6], 'AR_TLAI': [4, 5, 6, 7, 8]}
tpu25ratio           {'FGEV': [4, 8], 'TSOI_10CM': [4], 'TLAI': [4, 8, 9], 'FSR': [4], 'FSH': [4], 'FCTR_TLAI': [4]}
non_boreal_peatfire_c {'FAREA_BURNED': [1], 'COL_FIRE_CLOSS_TLAI': [1]}
prh30                {'FAREA_BURNED': [9], 'COL_FIRE_CLOSS_TLAI': [8, 9]}
psi50                {'TSOI_10CM': [1]}
KCN                  {'GPP_TLAI': [9], 'AR_TLAI': [9]}
slopebeta            {'QOVER': [6], 'SNOWDP': [9], 'FSH': [9]}
slopemax             {'QOVER': [6, 7, 8]}


In [134]:
def make_list_biome(datavars,nx,biomes):
    domain = 'biome'
    ens_name = 'CTL2010'
    out=dict()
    for datavar in datavars:
        f = './data/'+ens_name+'_'+datavar+'_'+domain+'.nc'
        ds = xr.open_dataset(f)
        for b in biomes:
            da = ds[datavar+'_mean'].isel(biome=b)
            p = ds.param
            m = ds.minmax
            xmins,xmaxs,pvals = top_n(da,nx,p,m)
            for p in pvals:
                if p not in out:
                    out[p] = {datavar:[b]}
                elif datavar not in out[p]:
                    out[p][datavar] = [b]
                else:
                    out[p][datavar] = [*out[p][datavar],b]
    return out

In [150]:
cb2m5 = make_list_biome(datavars,5,[2])
cb8m5 = make_list_biome(datavars,5,[8])
b2 = str(ds.biome_name[2].values)[:-8]
b8 = str(ds.biome_name[8].values)
comp_lists(cb2m5,cb8m5,b2,b8)

    Tropical seasonal forest       Boreal forest
-------------------------------------------------------------
1   clay_pf                        clay_pf
2   cv                             cv
3   d_max                          d_max
4   decomp_depth_efolding          decomp_depth_efolding
5   dleaf                          dleaf
6   fff                            fff
7   froot_leaf                     froot_leaf
8   grperc                         grperc
9   jmaxb0                         jmaxb0
10  kmax                           kmax
11  leaf_long                      leaf_long
12  liq_canopy_storage_scalar      liq_canopy_storage_scalar
13  maximum_leaf_wetted_fraction   maximum_leaf_wetted_fraction
14  medlynintercept                medlynintercept
15  medlynslope                    medlynslope
16  q10_mr                         q10_mr
17  r_mort                         r_mort
18  rf_s2s3_bgc                    rf_s2s3_bgc
19  rholnir                        rholnir
20  rholvis       

In [176]:
datavars = mainvars+cpools+fluxvars
exps = ['C285','C867','AF1855','AF2095','NDEP']
nx = 3
pgm3 = make_list_pert(datavars,nx,exps)
comp_lists(cgm5,pgm3,'ctl-global-mean','perturb-global-mean')

    ctl-global-mean                perturb-global-mean
-------------------------------------------------------------
1   FUN_fracfixers                 FUN_fracfixers
2   accum_factor                   accum_factor
3   baseflow_scalar                baseflow_scalar
4   clay_pf                        clay_pf
5   cv                             cv
6   d_max                          d_max
7   decomp_depth_efolding          decomp_depth_efolding
8   fff                            fff
9   frac_sat_soil_dsl_init         frac_sat_soil_dsl_init
10  froot_leaf                     froot_leaf
11  jmaxb0                         jmaxb0
12  jmaxb1                         jmaxb1
13  kmax                           kmax
14  liq_canopy_storage_scalar      liq_canopy_storage_scalar
15  lmrha                          lmrha
16  maximum_leaf_wetted_fraction   maximum_leaf_wetted_fraction
17  medlynintercept                medlynintercept
18  medlynslope                    medlynslope
19  om_frac_sf          

In [177]:
bps = set(pgm3)-set(cgm5)
for p in bps:
    print(p.ljust(20),pgm3[p])

q10_mr               {'FGEV': ['AF2095'], 'FSR': ['AF2095'], 'TOTVEGC': ['AF1855', 'AF2095'], 'TOTSOMC_1m': ['AF2095', 'NDEP'], 'FCEV': ['AF2095']}
e_ice                {'SOILWATER_10CM': ['AF1855'], 'SNOWDP': ['C867', 'NDEP']}
kn_nonmyc            {'NPP': ['C285'], 'AR': ['NDEP']}
maxpsi_hr            {'SCA_JJA': ['AF1855']}
jmaxhd               {'FAREA_BURNED': ['AF2095'], 'FSR': ['AF2095'], 'SCA_JJA': ['AF2095']}
leafcn               {'SCA_DJF': ['AF1855']}
tpuse_sf             {'FAREA_BURNED': ['C867', 'AF1855', 'AF2095'], 'FGEV': ['C867', 'AF2095'], 'QOVER': ['C867', 'AF2095'], 'QDRAI': ['C867', 'AF2095'], 'SOILWATER_10CM': ['C867'], 'TSOI_10CM': ['AF2095'], 'TLAI': ['C867', 'AF1855', 'AF2095'], 'FSR': ['AF2095'], 'FSH': ['C867', 'AF2095'], 'SCA_JJA': ['AF2095'], 'TOTVEGC': ['C867', 'AF2095'], 'GPP': ['AF2095'], 'NPP': ['AF2095'], 'AR': ['C867', 'AF2095'], 'HR': ['AF2095'], 'COL_FIRE_CLOSS': ['C867'], 'FCEV': ['AF2095'], 'FCTR': ['C867', 'AF1855', 'AF2095']}
tpu25ratio           {

In [179]:
datavars = mainvars+cpools+fluxvars
nx=5
c285gm5 = make_list_pert(datavars,nx,['AF1855'])
c867gm5 = make_list_pert(datavars,nx,['AF2095'])
comp_lists(c285gm5,c867gm5,'AF1855','AF2095')

    AF1855                         AF2095
-------------------------------------------------------------
1   accum_factor                   accum_factor
2   baseflow_scalar                baseflow_scalar
3   clay_pf                        clay_pf
4   d_max                          d_max
5   fff                            fff
6   frac_sat_soil_dsl_init         frac_sat_soil_dsl_init
7   jmaxhd                         jmaxhd
8   kcha                           kcha
9   krmax                          krmax
10  liq_canopy_storage_scalar      liq_canopy_storage_scalar
11  lmrhd                          lmrhd
12  lmrse                          lmrse
13  medlynintercept                medlynintercept
14  medlynslope                    medlynslope
15  n_baseflow                     n_baseflow
16  nstem                          nstem
17  q10_mr                         q10_mr
18  rhosnir                        rhosnir
19  sand_pf                        sand_pf
20  slopebeta                      sl

In [180]:
datavars = mainvars+cpools+fluxvars
cgm5 = make_list(datavars,'CTL2010','global',5,'mean')

In [182]:
cgi3 = make_list(datavars,'CTL2010','global',3,'iav')

In [183]:
exps = ['C285','C867','AF1855','AF2095','NDEP']
pgm3 = make_list_pert(datavars,3,exps)

In [184]:
datavars.remove('SCA_JJA')
datavars.remove('SCA_DJF')
datavars.remove('ALTMAX')
cbm3 = make_list_biome(datavars,3,1+np.arange(9))

In [192]:
skips = ['kc_nonmyc',
        'kn_nonmyc',
        'akc_active',
        'akn_active',
        'ekc_active',
        'ekn_active',
        'lf_fcel',
        'lf_flab',
        'lf_flig',
        'fr_fcel',
        'fr_flab',
        'fr_flig']

In [195]:
print('STRAWMAN')
print(10*'-'+'global-mean'+60*'-')
i=0
for p in sorted(cgm5):
    i+=1
    if p not in skips:
        print(str(i).ljust(3),p)

print(10*'-'+'global-iav'+60*'-')
x = set(cgi3)-set(cgm5)
for p in sorted(x):
    i+=1
    if p not in skips:
        print(str(i).ljust(3),p)        
        
print(10*'-'+'perturbations'+60*'-')
x = set(pgm3)-set(cgm5)-set(cgi3)
for p in sorted(x):
    i+=1
    if p not in skips:
        print(str(i).ljust(3),p)
print(10*'-'+'biomes'+60*'-')
x = set(cbm3)-set(pgm3)-set(cgm5)-set(cgi3)
for p in sorted(x):
    i+=1
    if p not in skips:
        print(str(i).ljust(3),p)


STRAWMAN
----------global-mean------------------------------------------------------------
1   FUN_fracfixers
2   accum_factor
3   baseflow_scalar
4   ceta
5   clay_pf
6   cv
7   d_max
8   decomp_depth_efolding
9   dleaf
10  fff
11  frac_sat_soil_dsl_init
12  fsr_pft
13  jmaxb0
14  jmaxb1
15  kmax
16  leaf_long
17  leafcn
18  liq_canopy_storage_scalar
19  lmr_intercept_atkin
20  maximum_leaf_wetted_fraction
21  medlynintercept
22  medlynslope
23  nstem
24  om_frac_sf
25  r_mort
26  rf_s2s3_bgc
27  rholnir
28  rholvis
29  rhosnir
30  sand_pf
31  snw_rds_refrz
32  tau_s2
33  tau_s3
34  taulnir
35  tkd_sand
36  upplim_destruct_metamorph
37  wc2wjb0
38  zetamaxstable
39  zsno
----------global-iav------------------------------------------------------------
40  fd_pft
41  theta_cj
42  wind_snowcompact_fact
----------perturbations------------------------------------------------------------
43  KCN
44  e_ice
45  fbw
46  froot_leaf
47  jmaxhd
48  kcha
50  lmrha
51  lmrhd
52  lmrse
53  maxpsi_hr

In [166]:
def make_list_pert(datavars,nx,exps):
    out = dict()
    for datavar in datavars:
        dv = datavar+'_mean'
        f0  = './data/CTL2010_'+datavar+'_'+domain+'.nc'
        ctl = xr.open_dataset(f0)

        for exp in exps:
            f   = './data/'+exp+'_'+datavar+'_'+domain+'.nc'
            ds  = xr.open_dataset(f)

            da = ds[dv]/ctl[dv]
            p = ds.param
            m = ds.minmax
            xmins,xmaxs,pvals = top_n(da,nx,p,m)

            for p in pvals:
                if p not in out:
                    out[p] = {datavar:[exp]}
                elif datavar not in out[p]:
                    out[p][datavar] = [exp]
                else:
                    out[p][datavar] = [*out[p][datavar],exp]
    return out

In [162]:
out

{'jmaxb0': {'FAREA_BURNED': ['C285', 'C867']},
 'clay_pf': {'FAREA_BURNED': ['C285']},
 'sand_pf': {'FAREA_BURNED': ['C285', 'C867'], 'FGEV': ['C285', 'C867']},
 'tpuse_sf': {'FAREA_BURNED': ['C867'], 'FGEV': ['C867']},
 'KCN': {'FGEV': ['C285']},
 'kmax': {'FGEV': ['C285', 'C867']}}

In [93]:
def print_dict(d,name):

    print(name)
    print('-'*100)
    
    nv = 20
    nx = [len(d[p]) for p in d]
    ix = np.flipud(np.argsort(nx))

    ps = np.array([p for p in d])
    nc = [len(p) for p in ps]
    nc = max(nc)

    for p in ps[ix]:

        vstrs=[]
        i=0
        vstr =''
        for v in d[p]:
            i+=1
            vstr = vstr+(v[:-5]).ljust(nv)
            if i==4:
                vstrs.append(vstr)
                i=0
                vstr=''
        if i!=0:
            vstrs.append(vstr)

        i=0
        for vstr in vstrs:
            i+=1
            if i==1:
                print(p.ljust(nc+2)+'| ',vstr)
            else:
                print(' '*(nc+2)+'| ',vstr)




In [62]:
ix

array([ 0, 22, 23, 24, 25, 26, 27, 28, 42, 33, 35, 36, 37, 38, 39, 40, 41,
       34, 20, 43, 17, 16, 15, 14, 19, 10, 32, 30, 29, 18,  2,  3, 12,  9,
        8,  7,  6,  1, 31, 11, 21, 13,  4,  5])

In [3]:
ens_name  = 'CTL2010'
domain    = 'global'
files     = glob.glob('./data/'+ens_name+'*'+domain+'*')
data_vars = [f.split(ens_name)[1].split(domain)[0][1:-1] for f in files]

In [4]:
glob_list = []
for f,v in zip(files,data_vars):
    ds = xr.open_dataset(f)
    for field in ds.data_vars:
        if v in field:
            da = ds[field]
            if 'iav' in field:
                nx = 5
            else:
                nx = 5
            xmins,xmaxs,pvals = top_n(da,nx,ds.param,ds.minmax)
            for p in pvals:
                if p not in glob_list:
                    glob_list.append(p)

In [5]:
domain    = 'TLAI'
altfiles  = glob.glob('./data/'+ens_name+'_*_'+domain+'*')
altvars   = [f.split(ens_name)[1].split(domain)[0][1:-1] for f in altfiles]
domain    = 'NPP'
altfiles2 = glob.glob('./data/'+ens_name+'_*_'+domain+'*')
altvars2  = [f.split(ens_name)[1].split(domain)[0][1:-1] for f in altfiles2]
altfiles = altfiles+altfiles2
altvars  = altvars+altvars2

In [6]:
normfiles = files.copy()
normvars  = data_vars.copy()
domain = 'global'
for altvar in altvars:
    for f,v in zip(normfiles,normvars):
        if altvar==v:
            normfiles.remove(f)
            normvars.remove(v)
normfiles = normfiles+altfiles
normvars  = normvars+altvars

In [7]:
glob_list2 = []
for f,v in zip(normfiles,normvars):
    ds = xr.open_dataset(f)
    for field in ds.data_vars:
        if v in field:
            da = ds[field]
            if 'iav' in field:
                nx = 5
            else:
                nx = 5
            xmins,xmaxs,pvals = top_n(da,nx,ds.param,ds.minmax)
            for p in pvals:
                if p not in glob_list2:
                    glob_list2.append(p)

In [8]:
len(glob_list2)

55

In [9]:
print('    standard'.ljust(34),'normalize by TLAI or NPP')
print('-'*65)
i = 0
for p in sorted(glob_list):
    if p in glob_list2:
        i+=1
        print(str(i).ljust(3),p.ljust(30),p)

print('-'*65)
              
uniqs1 = list(set(glob_list)-set(glob_list2))
uniqs2 = list(set(glob_list2)-set(glob_list))
uniqs1 = uniqs1+['']*(len(uniqs2)-len(uniqs1))

for p1,p2 in zip(uniqs1,uniqs2):
    i+=1
    print(str(i).ljust(3),p1.ljust(30),p2)


    standard                       normalize by TLAI or NPP
-----------------------------------------------------------------
1   FUN_fracfixers                 FUN_fracfixers
2   accum_factor                   accum_factor
3   baseflow_scalar                baseflow_scalar
4   ceta                           ceta
5   clay_pf                        clay_pf
6   csol_sand                      csol_sand
7   cv                             cv
8   d_max                          d_max
9   decomp_depth_efolding          decomp_depth_efolding
10  dleaf                          dleaf
11  fd_pft                         fd_pft
12  fff                            fff
13  frac_sat_soil_dsl_init         frac_sat_soil_dsl_init
14  fsr_pft                        fsr_pft
15  jmaxb0                         jmaxb0
16  jmaxb1                         jmaxb1
17  kcha                           kcha
18  kmax                           kmax
19  krmax                          krmax
20  leaf_long                    

### biome strawman

In [16]:
ens_name = 'CTL2010'
domain = 'biome'
files = glob.glob('./data/CTL2010*biome*')
data_vars = [f.split(ens_name)[1].split(domain)[0][1:-1] for f in files]

In [29]:
blist = []
nx = 3
for f,v in zip(files,data_vars):
    ds = xr.open_dataset(f)
    for bb in 1+np.arange(9):
        da = ds[v+'_mean'].isel(biome=bb)
        xmins,xmaxs,pvals = top_n(da,nx,ds.param,ds.minmax)
        for p in pvals:
            if p not in blist:
                print(v,bb,p)
                blist.append(p)

TWS 1 fff
TWS 1 baseflow_scalar
TWS 1 sand_pf
TWS 3 krmax
TWS 5 clay_pf
TWS 8 medlynintercept
TWS 9 om_frac_sf
COL_FIRE_CLOSS 1 psi50
COL_FIRE_CLOSS 1 kmax
COL_FIRE_CLOSS 1 decomp_depth_efolding
COL_FIRE_CLOSS 2 jmaxb0
COL_FIRE_CLOSS 2 FUN_fracfixers
COL_FIRE_CLOSS 6 jmaxb1
COL_FIRE_CLOSS 8 prh30
TOTVEGC 1 nstem
TOTVEGC 2 medlynslope
TOTVEGC 4 tpu25ratio
TOTVEGC 6 r_mort
TLAI 1 leaf_long
TLAI 2 lmr_intercept_atkin
TLAI 5 lmrha
AR 1 leafcn
AR 3 d_max
QDRAI 4 maximum_leaf_wetted_fraction
QDRAI 9 upplim_destruct_metamorph
TV 1 liq_canopy_storage_scalar
TV 3 vcmaxhd
TV 3 kcha
TV 6 zsno
TV 7 rhosnir
TV 7 zlnd
TV 9 zetamaxstable
QOVER 1 n_baseflow
QOVER 6 slopebeta
QOVER 6 slopemax
QHR 1 KCN
QHR 8 e_ice
FSR 1 rholvis
FSR 1 rholnir
FSR 1 taulnir
FSR 9 snw_rds_refrz
FGEV 1 frac_sat_soil_dsl_init
SNOWDP 1 fun_cn_flex_b
SNOWDP 2 wind_snowcompact_fact
SNOWDP 2 accum_factor
TOTSOMC_1m 1 tau_s3
TOTSOMC_1m 1 q10_mr
TOTSOMC_1m 5 rf_s2s3_bgc
TOTSOMC_1m 7 minpsi_hr
FCEV 1 cv
FCEV 2 dleaf
FAREA_BURNED 1

In [32]:
l1 = glob_list
l2 = blist
print('    standard'.ljust(34),'biome list')
print('-'*65)
i = 0
for p in sorted(l1):
    if p in l2:
        i+=1
        print(str(i).ljust(3),p.ljust(30),p)

print('-'*65)
              
uniqs1 = list(set(l1)-set(l2))
uniqs2 = list(set(l2)-set(l1))
uniqs1 = uniqs1+['']*(len(uniqs2)-len(uniqs1))

for p1,p2 in zip(uniqs1,uniqs2):
    i+=1
    print(str(i).ljust(3),p1.ljust(30),p2)

    standard                       biome list
-----------------------------------------------------------------
1   FUN_fracfixers                 FUN_fracfixers
2   accum_factor                   accum_factor
3   baseflow_scalar                baseflow_scalar
4   clay_pf                        clay_pf
5   cv                             cv
6   d_max                          d_max
7   decomp_depth_efolding          decomp_depth_efolding
8   dleaf                          dleaf
9   fff                            fff
10  frac_sat_soil_dsl_init         frac_sat_soil_dsl_init
11  jmaxb0                         jmaxb0
12  jmaxb1                         jmaxb1
13  kcha                           kcha
14  kmax                           kmax
15  krmax                          krmax
16  leaf_long                      leaf_long
17  leafcn                         leafcn
18  liq_canopy_storage_scalar      liq_canopy_storage_scalar
19  lmr_intercept_atkin            lmr_intercept_atkin
20  maximum_le